In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import train_test_split
load("~/conjecturing/sage/conjecturing.py")

/tmp/ipykernel_795849/3289088492.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Specify output files.

In [2]:
inv_file = open("2022_12_07_inv.txt", "w")
prop_file = open("2022_12_07_prop.txt", "w")

Specify the number of examples to use for conjecturing and skips.

In [3]:
num_train = 100
my_skips = 0.3

Read data.  

In [4]:
my_data =pd.read_excel("essi.xlsx",
                    header=int(0),
                    sheet_name = "Sheet1"   
                    )
print(my_data.shape)
my_data.head()

(25808, 63)


,Year,DSA,Program_Year.x,Facility_Count,HIE_B_Count,Weighted_HIE_DSA,HIE_DSA_Penetration,Time,ZIP,County.Code,...,Tx_R_O1,Tx_R_ST1,Tx_R_ST2,Flagging_ratio,Tx_R_ST_C,M_R_O1,M_R_ST1,HalfYear.y,HIE_DSA_P,P_HIE_Adopt
0,2012,ALOB,2013,102,0,0.0,0,1,35010,1037,...,0.05,1,1,1.0,1,0.08,1,7,0,0
1,2012,ALOB,2013,102,0,0.0,0,1,36033,1013,...,0.05,1,1,1.0,1,0.08,1,7,0,0
2,2012,ALOB,2013,102,0,0.0,0,1,35401,1125,...,0.05,1,1,1.0,1,0.08,1,7,0,0
3,2012,ALOB,2013,102,0,0.0,0,1,36401,1035,...,0.05,1,1,1.0,1,0.08,1,7,0,0
4,2012,ALOB,2013,102,0,0.0,0,1,35462,1063,...,0.05,1,1,1.0,1,0.08,1,7,0,0


Check the data types of the columns.  For categorical data, make sure the type is integer or objects.  Make sure the categories do not contain special characters besides numbers and "_".

In [5]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25808 entries, 0 to 25807
Data columns (total 63 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 25808 non-null  int64  
 1   DSA                  25808 non-null  object 
 2   Program_Year.x       25808 non-null  int64  
 3   Facility_Count       25808 non-null  int64  
 4   HIE_B_Count          25808 non-null  int64  
 5   Weighted_HIE_DSA     25808 non-null  float64
 6   HIE_DSA_Penetration  25808 non-null  int64  
 7   Time                 25808 non-null  int64  
 8   ZIP                  25808 non-null  int64  
 9   County.Code          25808 non-null  int64  
 10  State.Name           25808 non-null  object 
 11  County.Name          25808 non-null  object 
 12  County.FIPS          25808 non-null  int64  
 13  Provider_Number      25808 non-null  int64  
 14  Network              25808 non-null  int64  
 15  Facility_Name        25808 non-null 

Pandas thinks PTSURV_T and PT_HOS_T are integers/numeric, but they are categorical.  Recast them as objects.

In [6]:
#"PTSURV_T", "PT_HOS_T"
my_data = my_data.astype({"PTSURV_T": object, "PT_HOS_T": object})


In [7]:
my_data.dropna(subset=["PTSURV_T"], inplace=True)
#my_data.dropna().reset_index(drop=True)

Identify invariant and categorical columns and the target column.  The target should be in one of the lists.

In [8]:
#invariant_names=["Tx_R_O1", "N_Tx_Ctr1", "N_R_Ctr1", "N_Center", "M_R_O1", "M_R_ST1",   "N_PT_TRANS_S", "PT_TRANS_T1", "N_PTSURV_SUM", "N_PT_HOS_S", "VAVF_F", "N_DP_HGBD12", "N_DP_HGBD", "HOMEHD", "PD", "HD", "TOTSTAS"]
invariant_names=["Weighted_HIE_DSA", "Flagging_ratio", "Tx_R_O1", "N_Tx_Ctr1"]

categorical_names=["PTSURV_T", "PT_HOS_T"]
target = "PTSURV_T"

Generic code starts here.  Rename target column.  Select columns.

In [9]:
if target in categorical_names:
    categorical_names[categorical_names.index(target)] = "TARGET"
else:
    invariant_names[invariant_names.index(target)] = "TARGET"

my_data = my_data.rename(columns={target: "TARGET"})
print(my_data.columns)
my_data = my_data[invariant_names + categorical_names]
print(categorical_names)

Index(['Year', 'DSA', 'Program_Year.x', 'Facility_Count', 'HIE_B_Count',
       'Weighted_HIE_DSA', 'HIE_DSA_Penetration', 'Time', 'ZIP', 'County.Code',
       'State.Name', 'County.Name', 'County.FIPS', 'Provider_Number',
       'Network', 'Facility_Name', 'Unnamed: 16', 'City', 'STATE',
       'Late.Shift', 'Chain.Owned', 'Chain.Organization', 'TOTSTAS', 'HD',
       'PD', 'HOMEHD', 'N_DP_HGBD', 'N_DP_HGBD12', 'PTSURV_C', 'County',
       'HDKTVPM12_F', 'VAVF_F', 'F_Star', 'PT_HOS_T', 'PT_HOS_T1', 'PT_HOS_T2',
       'PT_HOS_C', 'N_PT_HOS_S', 'N_PTSURV_SUM', 'SHR', 'SMR', 'PT_TRANS_T',
       'PT_TRANS_T1', 'PT_TRANS_T2', 'N_PT_TRANS_S', 'TARGET', 'PTSURV_T1',
       'PTSURV_T2', 'R_Year', 'HalfYear.x', 'N_Tx_Ctr1', 'N_R_Ctr1',
       'N_Center', 'Tx_R_O1', 'Tx_R_ST1', 'Tx_R_ST2', 'Flagging_ratio',
       'Tx_R_ST_C', 'M_R_O1', 'M_R_ST1', 'HalfYear.y', 'HIE_DSA_P',
       'P_HIE_Adopt'],
      dtype='object')
['TARGET', 'PT_HOS_T']


Convert categorical variables to dummies.  One dummy for each binary variable and one dummy for each level for variables with more than two levels.

In [10]:
property_names = []
for col in categorical_names:
    if col != "TARGET":
        unique_vals=list(my_data[col].unique())  # if nan is a level
        #unique_vals=list(my_data[col].dropna().unique())  # if nan is not a level
        if len(unique_vals)==2: # just use one level for binary features
            property_names.append(col+"_"+str(unique_vals[1]))
        elif len(unique_vals) > 2: #one property for each level.
            for level in unique_vals:
                property_names.append(col+"_"+str(level))


if "TARGET" in categorical_names:
    target_property_names = []
    unique_vals = list(my_data["TARGET"].unique()) # if nan is a level
    #unique_vals = list(my_data["TARGET"].dropna().unique()) # if nan is not a level
    if len(unique_vals)==2:
        target_property_names.append("TARGET_"+str(unique_vals[1]))
    elif len(unique_vals) > 2:
        for level in unique_vals:
            target_property_names.append("TARGET_"+str(level))
            
my_df = pd.get_dummies(my_data, 
                       columns=categorical_names,
                       dtype=np.uint8,
                       dummy_na=True,  # False is the default.  If False, use dropna() above
                       drop_first=False) # False is the default

my_df = my_df.rename(lambda col: col.replace('.0', ''), axis='columns')
my_df.head()

,Weighted_HIE_DSA,Flagging_ratio,Tx_R_O1,N_Tx_Ctr1,TARGET_As Expected,TARGET_Better than Expected,TARGET_Not Available,TARGET_Worse than Expected,TARGET_nan,PT_HOS_T_As Expected,PT_HOS_T_Better than Expected,PT_HOS_T_Not Available,PT_HOS_T_Worse than Expected,PT_HOS_T_nan
0,0.0,1.0,0.05,157,1,0,0,0,0,1,0,0,0,0
1,0.0,1.0,0.05,157,1,0,0,0,0,0,1,0,0,0
2,0.0,1.0,0.05,157,0,0,0,1,0,1,0,0,0,0
3,0.0,1.0,0.05,157,1,0,0,0,0,1,0,0,0,0
4,0.0,1.0,0.05,157,1,0,0,0,0,1,0,0,0,0


Define class, invariants, properties, and target properties (if applicable).

In [11]:
class Example():
    def __init__(self, name, mydf):
        self.name = name
        self.mydf = mydf
        
for i in invariant_names:
    inv = build_inv(i)
    setattr(Example,inv.__name__,inv )

for i in property_names:
    prop = build_prop(i)
    setattr(Example, prop.__name__,prop)

if "TARGET" in categorical_names:
    for i in target_property_names:
        prop = build_prop(i)
        setattr(Example, prop.__name__, prop)
else:
    target_invariant = invariant_names.index("TARGET")
print(property_names)

['PT_HOS_T_As Expected', 'PT_HOS_T_Better than Expected', 'PT_HOS_T_Worse than Expected', 'PT_HOS_T_Not Available', 'PT_HOS_T_nan']


Split into training and testing data.

In [12]:
if "TARGET" in categorical_names:
    X_train, X_test = train_test_split(
        my_df.index,
        stratify=my_data["TARGET"],  # stratify on target levels
        train_size=num_train,
        random_state=12345
    )
else:
    X_train, X_test = train_test_split(
        my_df.index,
        train_size=num_train,
        random_state=12345
    )

Create examples for conjecturing.

In [13]:
train_examples = [Example(i, my_df) for i in X_train]
test_examples = [Example(i, my_df) for i in X_test]

Get lists of invariant and property functions.

In [14]:
invariants =[]
for i in invariant_names:
    invariants.append(Example.__dict__[i])
properties=[]
for i in property_names:
    properties.append(Example.__dict__[i])
target_properties=[]
if "TARGET" in categorical_names:
    for i in target_property_names:
        target_properties.append(Example.__dict__[i])


In [40]:
my_df

,Age,Fare,SibSp,Parch,TARGET_0,TARGET_1,TARGET_nan,Sex_female,Sex_male,Sex_nan,...,cabin_letter_D,cabin_letter_E,cabin_letter_F,cabin_letter_G,cabin_letter_T,cabin_letter_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,22.0,7.2500,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
2,38.0,71.2833,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,26.0,7.9250,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4,35.0,53.1000,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
5,35.0,8.0500,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,27.0,13.0000,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
888,19.0,30.0000,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
889,NaN,23.4500,1,2,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0


Invariant conjecturing - upper and lower bounds.

In [15]:
#define operators for expression tree to build upper bounsand lower bouns for each class
use_operators =  { '-1', '+1', '*2', '/2', '^2', '-()', '1/', 
                  'sqrt', 'ln', 'log10', 'exp', '10^', 'ceil', 
                  'floor', 'abs', '+', '*', 'max', 'min', '-', '/', '^'}

inv_conjectures = []

if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == True]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
    print(len(inv_conjectures))
    if len(target_property_names) == 1:
        value = target_property_names[0]
        print(value + " False")
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == False]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
else: # target is an invariant
    my_examples = [example for example in train_examples]
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators, 
                       upperBound=True, 
                       time=Integer(5)
                        # ,debug=True,
                        #  verbose=True,
                        ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators,
                       upperBound=False, 
                       time=Integer(5)
                       ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs     
print(len(inv_conjectures))  

for c in inv_conjectures:
    inv_file.write("%s\n" % c)
    inv_file.flush()
inv_file.close()


TARGET_As Expected
TARGET_Worse than Expected
TARGET_Better than Expected
TARGET_Not Available
353
353


Property conjecturing - sufficient conditions for a categorical target values.  For a binary target, get sufficient conditions for the positive class and necessary conditions for the negative class.

In [17]:
all_properties = ["TARGET"] + properties + inv_conjectures #"TARGET" is just a placeholder
prop_conjs = []
conditions = {}
if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        all_properties[0] = Example.__dict__[value]
        #print(all_properties)
        these_prop_conjs = propertyBasedConjecture(objects=train_examples, # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           #verbose=True,
                                           #debug=True,
                                           skips=my_skips)
        conditions[value] = []
        for c in these_prop_conjs: # edit here 6/27/23 just get premises once
            conditions[value].append(get_premise(c, myprint=False))
        prop_conjs += these_prop_conjs
    if len(target_property_names) == 1:
        print(value + " Necessary")
        all_properties[0] = Example.__dict__[value]
        these_prop_conjs = propertyBasedConjecture(objects=train_examples,  # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           sufficient=False,
                                           #verbose=True,
                                            #  debug=True,
                                             skips=my_skips)
        conditions["necessary"] = []
        for c in these_prop_conjs:
            conditions["necessary"].append(get_conclusion(c, myprint=False))
        prop_conjs += these_prop_conjs  # edit here 6/27/23
        
for c in prop_conjs:
    prop_file.write("%s\n" % convert_name_back(c.__name__))
    prop_file.flush()
    
prop_file.close()

TARGET_As Expected


/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: divide by zero encountered in log
  return ln(args[0], **kwds)
/lustre/home/clarson/conjecturing/sage/conjecturing.py:267: RuntimeWarning: overflow encountered in scalar multiply
  return (lambda x: x*x), 1
/lustre/home/clarson/conjecturing/sage/conjecturing.py:177: RuntimeWarning: invalid value encountered in scalar multiply
  stack.append(op(left, right))
/lustre/home/clarson/conjecturing/sage/conjecturing.py:279: RuntimeWarning: overflow encountered in scalar power
  return (lambda x: 10**x), 1
/lustre/home/clarson/conjecturing/sage/conjecturing.py:132: RuntimeWarning: overflow encountered in exp
  stack.append(op(stack.pop()))
/lustre/home/clarson/conjecturing/sage/conjecturing.py:177: RuntimeWarning: overflow encountered in scalar power
  stack.append(op(left, right))
/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: 

(~(N_Tx_Ctr1_leq_inverse_of_2_times_e_to_the_power_open_bracket_2_times_e_to_the_power_open_bracket_10_to_the_power_Weighted_HIE_DSA_close_bracket_plus_1_close_bracket))->(TARGET_AsExpected)
(~(N_Tx_Ctr1_geq_2_times_ceilopen_bracket_10_to_the_power_open_bracket_10_to_the_power_Tx_R_O1_close_bracket_plus_1_close_bracket))->(TARGET_AsExpected)
(~(Weighted_HIE_DSA_leq_open_bracket_Flagging_ratio_plus_Tx_R_O1_close_bracket_divided_by_open_bracket_logopen_bracket_N_Tx_Ctr1_close_bracket_minus_1_close_bracket))->(TARGET_AsExpected)
((Tx_R_O1_geq_10_to_the_power_open_bracket_10_to_the_power_open_bracket_Weighted_HIE_DSA_minus_1_close_bracket_minus_1_close_bracket_minus_Flagging_ratio)^(Weighted_HIE_DSA_leq__minus_inverse_of_open_bracket_sqrtopen_bracket_N_Tx_Ctr1_close_bracket_plus_1_close_bracket_plus_Tx_R_O1))->(TARGET_AsExpected)
(~(N_Tx_Ctr1_geq_e_to_the_power_open_bracket_e_to_the_power_open_bracket_e_to_the_power_open_bracket_Weighted_HIE_DSA_to_the_power_open_bracket_inverse_of_4_close

In [18]:
len(these_prop_conjs)

1

Apply conjectures to train and test data if target is categorical.

In [19]:
X_train_df = my_df.loc[X_train,property_names+invariant_names]  # drop target and one level for each binary variable
X_test_df = my_df.loc[X_test,property_names+invariant_names]
y_train_df = my_data.loc[X_train,"TARGET"] # get original target, even if it is multiple levels
y_test_df = my_data.loc[X_test, "TARGET"]
if "TARGET" in categorical_names:
    for value in target_property_names:
        for i, condition in enumerate(conditions[value]):
            X_train_df['conj_' + str(i)] = [condition(example) for example in train_examples]
            X_test_df['conj_' + str(i)] = [condition(example) for example in test_examples]
    if len(target_property_names) == 1:
        for i, condition in enumerate(conditions["necessary"]):
            X_train_df['conj_' + str(i)] = [condition(example) for example in train_examples]
            X_test_df['conj_' + str(i)] = [condition(example) for example in test_examples]
        
    X_train_df.head()
    #y_train_df.head()    
    print(conditions)

/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: divide by zero encountered in log
  return ln(args[0], **kwds)
/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: divide by zero encountered in log
  return ln(args[0], **kwds)
/lustre/home/clarson/conjecturing/sage/conjecturing.py:279: RuntimeWarning: overflow encountered in scalar power
  return (lambda x: 10**x), 1


{'TARGET_As Expected': [~N_Tx_Ctr1_leq_inverse_of_2_times_e_to_the_power_open_bracket_2_times_e_to_the_power_open_bracket_10_to_the_power_Weighted_HIE_DSA_close_bracket_plus_1_close_bracket, ~N_Tx_Ctr1_geq_2_times_ceilopen_bracket_10_to_the_power_open_bracket_10_to_the_power_Tx_R_O1_close_bracket_plus_1_close_bracket, ~Weighted_HIE_DSA_leq_open_bracket_Flagging_ratio_plus_Tx_R_O1_close_bracket_divided_by_open_bracket_logopen_bracket_N_Tx_Ctr1_close_bracket_minus_1_close_bracket, Tx_R_O1_geq_10_to_the_power_open_bracket_10_to_the_power_open_bracket_Weighted_HIE_DSA_minus_1_close_bracket_minus_1_close_bracket_minus_Flagging_ratio^Weighted_HIE_DSA_leq__minus_inverse_of_open_bracket_sqrtopen_bracket_N_Tx_Ctr1_close_bracket_plus_1_close_bracket_plus_Tx_R_O1, ~N_Tx_Ctr1_geq_e_to_the_power_open_bracket_e_to_the_power_open_bracket_e_to_the_power_open_bracket_Weighted_HIE_DSA_to_the_power_open_bracket_inverse_of_4_close_bracket_close_bracket_close_bracket_close_bracket, N_Tx_Ctr1_geq_open_brack

Calculate support, precision, and lift.

In [20]:
target_property_names

['TARGET_As Expected',
 'TARGET_Worse than Expected',
 'TARGET_Better than Expected',
 'TARGET_Not Available']

In [21]:
support = []
lift = []
precision = []
if "TARGET" in categorical_names:
    for value in target_property_names:
        print("value: {}".format(value))
        my_function = getattr(Example, value)
        for i, condition in enumerate(conditions[value]):
            print("condition: {}".format(condition))
            num_true = 0
            num_in_class = 0
            num_hit = 0
            for example in test_examples:
                if condition(example) == True:
                    num_true += 1
                    if my_function(example) == True:
                        num_hit += 1
                if my_function(example) == True:
                    num_in_class += 1
            support.append(num_true)
            if num_hit > 0: 
                precision.append(n(num_hit/num_true))
                lift.append(n(num_hit/num_true)/n(num_in_class/len(test_examples)))
            else:
                precision.append(0.0)
                lift.append(0.0)
    if len(target_property_names) == 1:
        for i, condition in enumerate(conditions["necessary"]):
            print("condition: {}".format(condition))
            num_false = 0
            num_in_class = 0
            num_hit = 0
            for example in test_examples:
                if condition(example) == False:
                    num_false += 1
                    if my_function(example) == False:
                        num_hit += 1
                if my_function(example) == False:
                    num_in_class += 1
            support.append(num_false)
            if num_hit > 0: 
                precision.append(n(num_hit/num_false))
                lift.append(n(num_hit/num_false)/n(num_in_class/len(test_examples)))
            else:
                precision.append(0.0)
                lift.append(0.0)
            
results_df = pd.DataFrame({'support':support, 'precision':precision, 'lift':lift})
        
results_df

value: TARGET_As Expected
condition: ~N_Tx_Ctr1_leq_inverse_of_2_times_e_to_the_power_open_bracket_2_times_e_to_the_power_open_bracket_10_to_the_power_Weighted_HIE_DSA_close_bracket_plus_1_close_bracket
condition: ~N_Tx_Ctr1_geq_2_times_ceilopen_bracket_10_to_the_power_open_bracket_10_to_the_power_Tx_R_O1_close_bracket_plus_1_close_bracket
condition: ~Weighted_HIE_DSA_leq_open_bracket_Flagging_ratio_plus_Tx_R_O1_close_bracket_divided_by_open_bracket_logopen_bracket_N_Tx_Ctr1_close_bracket_minus_1_close_bracket
condition: Tx_R_O1_geq_10_to_the_power_open_bracket_10_to_the_power_open_bracket_Weighted_HIE_DSA_minus_1_close_bracket_minus_1_close_bracket_minus_Flagging_ratio^Weighted_HIE_DSA_leq__minus_inverse_of_open_bracket_sqrtopen_bracket_N_Tx_Ctr1_close_bracket_plus_1_close_bracket_plus_Tx_R_O1
condition: ~N_Tx_Ctr1_geq_e_to_the_power_open_bracket_e_to_the_power_open_bracket_e_to_the_power_open_bracket_Weighted_HIE_DSA_to_the_power_open_bracket_inverse_of_4_close_bracket_close_bracket_

/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: divide by zero encountered in log
  return ln(args[0], **kwds)


condition: ~(Tx_R_O1_geq_open_bracket_inverse_of_open_bracket_10_to_the_power_open_bracket_10_to_the_power_Flagging_ratio_close_bracket_minus_logopen_bracket_N_Tx_Ctr1_close_bracket_close_bracket_close_bracket|Weighted_HIE_DSA_geq_open_bracket_inverse_of_2_times_Tx_R_O1_minus_1_close_bracket_to_the_power_ceilopen_bracket_inverse_of_2_times_N_Tx_Ctr1_close_bracket)
value: TARGET_Worse than Expected
condition: Flagging_ratio_geq__minus_open_bracket_Tx_R_O1_times_Weighted_HIE_DSA_minus_1_close_bracket_to_the_power_N_Tx_Ctr1&PT_HOS_T_WorsethanExpected
condition: N_Tx_Ctr1_geq_logopen_bracket_10_to_the_power_open_bracket_10_to_the_power_open_bracket_10_to_the_power_Tx_R_O1_close_bracket_minus_1_close_bracket_close_bracket^N_Tx_Ctr1_geq_2_times_ceilopen_bracket_10_to_the_power_open_bracket_10_to_the_power_Tx_R_O1_close_bracket_plus_1_close_bracket


/lustre/home/clarson/conjecturing/sage/conjecturing.py:279: RuntimeWarning: overflow encountered in scalar power
  return (lambda x: 10**x), 1


value: TARGET_Better than Expected
condition: ~N_Tx_Ctr1_geq__minus_inverse_of_open_bracket_Tx_R_O1_minus_4_times_Weighted_HIE_DSA_close_bracket
condition: ~Tx_R_O1_geq_4_times_open_bracket_Flagging_ratio_minus_1_close_bracket_squared_divided_by_N_Tx_Ctr1
condition: Weighted_HIE_DSA_geq_open_bracket_Tx_R_O1_minus_1_close_bracket_to_the_power_ceilopen_bracket_logopen_bracket_10_to_the_power_N_Tx_Ctr1_close_bracket_close_bracket^N_Tx_Ctr1_leq_10_to_the_power_flooropen_bracket_e_to_the_power_open_bracket_inverse_of_2_divided_by_Tx_R_O1_close_bracket_close_bracket
condition: ~Weighted_HIE_DSA_leq_absopen_bracket_logopen_bracket_sqrtopen_bracket_Tx_R_O1_close_bracket_close_bracket_squared_minus_1_close_bracket&PT_HOS_T_WorsethanExpected
value: TARGET_Not Available
condition: PT_HOS_T_NotAvailable


,support,precision,lift
0,4068,0.843903638151426,1.04258132200475
1,4144,0.839527027027027,1.03717433854634
2,2975,0.851764705882353,1.05229309716101
3,4142,0.824239497827137,1.01828771253496
4,2644,0.813161875945537,1.00460211960247
5,1709,0.826214160327677,1.02072726386198
6,1151,0.847958297132928,1.04759055709997
7,1540,0.782467532467532,0.966681499575920
8,5092,0.830322073841320,1.02580229104295
9,3534,0.805319750990379,0.994913746862447


In [22]:
support

[4068,
 4144,
 2975,
 4142,
 2644,
 1709,
 1151,
 1540,
 5092,
 3534,
 6840,
 2191,
 657,
 640,
 112,
 220,
 513,
 18,
 858]

In [23]:
precision

[0.843903638151426,
 0.839527027027027,
 0.851764705882353,
 0.824239497827137,
 0.813161875945537,
 0.826214160327677,
 0.847958297132928,
 0.782467532467532,
 0.830322073841320,
 0.805319750990379,
 0.813742690058480,
 0.850296668188042,
 0.168949771689498,
 0.112500000000000,
 0.169642857142857,
 0.172727272727273,
 0.105263157894737,
 0.0555555555555556,
 0.851981351981352]